# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Liu  ->  J. Liu  |  ['J. Liu']
R. Zhang  ->  R. Zhang  |  ['R. Zhang']
J. Shi  ->  J. Shi  |  ['J. Shi']
J. Li  ->  J. Li  |  ['J. Li']
D. Muley  ->  D. Muley  |  ['D. Muley']
H. Klahr  ->  H. Klahr  |  ['H. Klahr']
F. Walter  ->  F. Walter  |  ['F. Walter']
Arxiv has 84 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2405.02388
extracting tarball to tmp_2405.02388...

 done.
Retrieving document from  https://arxiv.org/e-print/2405.02863
extracting tarball to tmp_2405.02863... done.
Retrieving document from  https://arxiv.org/e-print/2405.02978


/tmp/ipykernel_2174/3009462391.py:49: LatexWarning: 2405.02978 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2405.03135
extracting tarball to tmp_2405.03135...

...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'threetemp_spiral_aanda_rev1_clean.bbl' from 'tmp_2405.03375/threetemp_spiral_aanda_rev1_clean.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Found 83 bibliographic references in tmp_2405.03375/threetemp_spiral_aanda_rev1_clean.bbl.
syntax error in line 581: premature end of file
Retrieving document from  https://arxiv.org/e-print/2405.03686
extracting tarball to tmp_2405.03686...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2405.03686/mnras_template.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'affiliations' from 'tmp_2405.03686/affiliations.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2405.03135-b31b1b.svg)](https://arxiv.org/abs/arXiv:2405.03135) | **CURLING - I. The Influence of Point-like Image Approximation on the  Outcomes of Cluster Strong Lens Modeling**  |
|| Y. Xie, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-05-07*|
|*Comments*| *12 pages, 8 figures*|
|**Abstract**| Cluster-scale strong lensing is a powerful tool for exploring the properties of dark matter and constraining cosmological models. However, due to the complex parameter space, pixelized strong lens modeling in galaxy clusters is computationally expensive, leading to the point-source approximation of strongly lensed extended images, potentially introducing systematic biases. Herein, as the first paper of the ClUsteR strong Lens modelIng for the Next-Generation observations (CURLING) program, we use lensing ray-tracing simulations to quantify the biases and uncertainties arising from the point-like image approximation for JWST-like observations. Our results indicate that the approximation works well for reconstructing the total cluster mass distribution, but can bias the magnification measurements near critical curves and the constraints on the cosmological parameters, the total matter density of the Universe $\Omega_{\rm m}$, and dark energy equation of state parameter $w$. To mitigate the biases, we propose incorporating the extended surface brightness distribution of lensed sources into the modeling. This approach reduces the bias in magnification from 46.2 per cent to 0.09 per cent for $\mu \sim 1000$. Furthermore, the median values of cosmological parameters align more closely with the fiducial model. In addition to the improved accuracy, we also demonstrate that the constraining power can be substantially enhanced. In conclusion, it is necessary to model cluster-scale strong lenses with pixelized multiple images, especially for estimating the intrinsic luminosity of highly magnified sources and accurate cosmography in the era of high-precision observations. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2405.03375-b31b1b.svg)](https://arxiv.org/abs/arXiv:2405.03375) | **Three-temperature radiation hydrodynamics with PLUTO: Thermal and  kinematic signatures of accreting protoplanets**  |
|| <mark>D. Muley</mark>, J. D. M. Fuksman, <mark>H. Klahr</mark> |
|*Appeared on*| *2024-05-07*|
|*Comments*| *Accepted to Astronomy and Astrophysics; 22 pages, 19 figures incl. Appendix. Comments and questions welcome*|
|**Abstract**| In circumstellar disks around young stars, the gravitational influence of nascent planets produces telltale patterns in density, temperature, and kinematics. To better understand these signatures, we first performed 3D hydrodynamical simulations of a 0.012 $M_{\odot}$ disk, with a Saturn-mass planet orbiting circularly in-plane at 40 au. We tested four different disk thermodynamic prescriptions (in increasing order of complexity, local isothermality, $\beta$-cooling, two-temperature radiation hydrodynamics, and three-temperature radiation hydrodynamics), finding that $\beta$-cooling offers a reasonable approximation for the three-temperature approach when the planet is not massive or luminous enough to substantially alter the background temperature and density structure. Thereafter, using the three-temperature scheme, we relaxed this assumption, simulating a range of different planet masses (Neptune-mass, Saturn-mass, Jupiter-mass) and accretion luminosities (0, $10^{-3} L_{\odot}$) in the same disk. Our investigation revealed that signatures of disk-planet interaction strengthen with increasing planet mass, with circumplanetary flows becoming prominent in the high-planet-mass regime. Accretion luminosity, which adds pressure support around the planet, was found to weaken the midplane Doppler-flip, potentially visible in optically thin tracers like C$^{18}$O, while strengthening the spiral signature, particularly in upper disk layers sensitive to thicker lines, like those of $^{12}$CO. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2405.03686-b31b1b.svg)](https://arxiv.org/abs/arXiv:2405.03686) | **JWST Observations of Starbursts: Cold Clouds and Plumes Launching in the  M82 Outflow**  |
|| D. B. Fisher, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2024-05-07*|
|*Comments*| *Submitted to MNRAS*|
|**Abstract**| In this paper we study the filamentary substructure of 3.3 $\mu$m PAH emission from JWST/NIRCam observations in the base of the M82 star-burst driven wind. We identify plume-like substructure within the PAH emission with widths of $\sim$50 pc. Several of the plumes extend to the edge of the field-of-view, and thus are at least 200-300 pc in length. In this region of the outflow, the vast majority ($\sim$70\%) of PAH emission is associated with the plumes. We show that those structures contain smaller scale "clouds" with widths that are $\sim$5-15 pc, and they are morphologically similar to the results of "cloud-crushing" simulations. We estimate the cloud-crushing time-scales of $\sim$0.5-3 Myr, depending on assumptions. We show this time scale is consistent with a picture in which these observed PAH clouds survived break-out from the disk rather than being destroyed by the hot wind. The PAH emission in both the midplane and the outflow is shown to tightly correlate with that of Pa$\alpha$ emission (from HST/NICMOS data), at the scale of both plumes and clouds, though the ratio of PAH-to-Pa$\alpha$ increases at further distances from the midplane. Finally, we show that the outflow PAH emission is suppressed in regions of the M82 wind that are bright in X-ray emission. Overall, our results are broadly consistent with a picture in which cold gas in galactic outflows is launched via hierarchically structured plumes, and those small scale clouds are more likely to survive the wind environment when collected into the larger plume structure. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2405.02388-b31b1b.svg)](https://arxiv.org/abs/arXiv:2405.02388) | **Gravothermal Catastrophe in Resonant Self-interacting Dark Matter Models**  |
|| V. Tran, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-05-07*|
|*Comments*| *10 pages, 6 figures, 1 table, submitted to PRD*|
|**Abstract**| We investigate a self-interacting dark matter (SIDM) model featuring a velocity-dependent cross section with an order-of-magnitude resonant enhancement of the cross section at $\sim 16\,{\rm km}\,{\rm s}^{-1}$. To understand the implications for the structure of dark matter halos, we perform N-body simulations of isolated dark matter halos of mass $\sim 10^8\,{\rm M}_\odot$, a halo mass selected to have a maximum response to the resonance. We track the core formation and the gravothermal collapse phases of the dark matter halo in this model and compare the halo evolving with the resonant cross section with halos evolving with velocity-independent cross sections. We show that dark matter halo evolution with the resonant cross section exhibits a deviation from universality that characterizes halo evolution with velocity-independent cross sections. The halo evolving under the influence of the resonance reaches a lower minimum central density during core formation. It subsequently takes about $20\%$ longer to reach its initial central density during the collapse phase. These results motivate a more detailed exploration of halo evolution in models with pronounced resonances. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2405.02863-b31b1b.svg)](https://arxiv.org/abs/arXiv:2405.02863) | **Stellar X-ray activity and habitability revealed by ROSAT sky survey**  |
|| H. Han, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-05-07*|
|*Comments*| *17 pages, 12 figures, accept for publish in ApJS*|
|**Abstract**| Using the homogeneous X-ray catalog from ROSAT observations, we conducted a comprehensive investigation into stellar X-ray activity-rotation relations for both single and binary stars. Generally, the relation for single stars consists of two distinct regions: a weak decay region, indicating a continued dependence of the magnetic dynamo on stellar rotation rather than a saturation regime with constant activity, and a rapid decay region, where X-ray activity is strongly correlated with the Rossby number. Detailed analysis reveals more fine structures within the relation: in the extremely fast rotating regime, a decrease in X-ray activity was observed with increasing rotation rate, referred to as super-saturation, while in the extremely slow rotating region, the relation flattens, mainly due to the scattering of F stars. This scattering may result from intrinsic variability in stellar activities over one stellar cycle or the presence of different dynamo mechanisms. Binaries exhibit a similar relation to that of single stars while the limited sample size prevented the identification of fine structures in the relation for binaries. We calculated the mass loss rates of planetary atmosphere triggered by X-ray emissions from host stars. Our findings indicate that for an Earth-like planet within the stellar habitable zone, it would easily lose its entire primordial H/He envelope (equating to about 1% of the planetary mass). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2405.02978-b31b1b.svg)](https://arxiv.org/abs/arXiv:2405.02978) | **Four-hundred Very Metal-poor Stars Studied with LAMOST and Subaru. III.  Dynamically Tagged Groups and Chemodynamical Properties**  |
|| <mark>R. Zhang</mark>, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2024-05-07*|
|*Comments*| *30 pages, 24 figures, published in ApJ*|
|**Abstract**| Very metal-poor (VMP) stars record the signatures of early accreted galaxies, making them essential tools for unraveling the early stages of Galaxy formation. Understanding the origin of VMP stars requires comprehensive studies of their chemical compositions and kinematics, which are currently lacking. Hence, we conduct a chemodynamical analysis of 352 VMP stars selected from one of the largest uniform high-resolution VMP star samples, jointly obtained from LAMOST and Subaru. We apply a friends-of-friends clustering algorithm to the master catalog of this high-resolution sample, which consists of 5778 VMP stars. It results in 131 dynamically tagged groups with 89 associated with known substructures in the Milky Way, including Gaia-Sausage-Enceladus (GSE), Thamnos, Helmi streams, Sequoia, Wukong, Pontus, and the very metal-poor disk (VMPD). Our findings are: (i) the VMPD shows lower Zn abundances than the rest, which indicates that it could be a relic of small stellar systems; (ii) Sequoia shows moderately high r-process abundances; (iii) Helmi streams show deficiencies in carbon and light neutron-capture elements; (iv) the fraction of carbon-enhanced metal-poor stars with no enhancement in heavy elements (CEMP-no stars) seems low in the VMPD and the Helmi streams; and (v) a subgroup in GSE exhibits a very high fraction of r-process enhanced stars, with four out of five showing [Eu/Fe]> +1.0. The abundance patterns of other elements in VMP substructures largely match the whole VMP sample. We also study large-scale correlations between abundance ratios and kinematics without classifying stars into substructures, but it does not yield significant correlations once the overall chemical evolution is considered for most elements. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2405.03135.md
    + _build/html/tmp_2405.03135/./fig5-2dmag.png
    + _build/html/tmp_2405.03135/./fig6-muarea.png
    + _build/html/tmp_2405.03135/./fig4-mass.png
exported in  _build/html/2405.03375.md
    + _build/html/tmp_2405.03375/./azi_cuts_integrated_spiral_cut_allmass_3t_scalar.png
    + _build/html/tmp_2405.03375/./azi_cuts_integrated_spiral_cut_allmass_3t_vector.png
    + _build/html/tmp_2405.03375/./vx1_diff_spiral_cut_allmass_3t.png
    + _build/html/tmp_2405.03375/./azi_cuts_integrated_spiral_cut_allmass_3t_scalar.png
    + _build/html/tmp_2405.03375/./azi_cuts_integrated_spiral_cut_allmass_3t_vector.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# CURLING - $\uppercase$$\expandafter{\romannumeral1}$. The Influence of Point-like Image Approximation on the Outcomes of Cluster Strong Lens Modeling

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2405.03135-b31b1b.svg)](https://arxiv.org/abs/2405.03135)<mark>Appeared on: 2024-05-07</mark> -  _12 pages, 8 figures_

</div>
<div id="authors">

Y. Xie, et al. -- incl., <mark>J. Li</mark>

</div>
<div id="abstract">

**Abstract:** Cluster-scale strong lensing is a powerful tool for exploring the properties of dark matter and constraining cosmological models. However, due to the complex parameter space, pixelized strong lens modeling in galaxy clusters is computationally expensive, leading to the point-source approximation of strongly lensed extended images, potentially introducing systematic biases. Herein, as the first paper of the ClUsteR strong Lens modelIng for the Next-Generation observations (CURLING) program, we use lensing ray-tracing simulations to quantify the biases and uncertainties arising from the point-like image approximation for JWST-like observations. Our results indicate that the approximation works well for reconstructing the total cluster mass distribution, but can bias the magnification measurements near critical curves and the constraints on the cosmological parameters, the total matter density of the Universe $\Omega_{\rm m}$ , and dark energy equation of state parameter $w$ . To mitigate the biases, we propose incorporating the extended surface brightness distribution of lensed sources into the modeling. This approach reduces the bias in magnification from 46.2 per cent to 0.09 per cent for $\mu \sim 1000$ . Furthermore, the median values of cosmological parameters align more closely with the fiducial model. In addition to the improved accuracy, we also demonstrate that the constraining power can be substantially enhanced. In conclusion, it is necessary to model cluster-scale strong lenses with pixelized multiple images, especially for estimating the intrinsic luminosity of highly magnified sources and accurate cosmography in the era of high-precision observations.

</div>

<div id="div_fig1">

<img src="tmp_2405.03135/./fig5-2dmag.png" alt="Fig5" width="100%"/>

**Figure 5. -** Log-scale difference between the recovered and fiducial magnification maps at $z_{\rm s} = 10$ with Lenstool(left panel) and pixelated modeling (right panel), with critical curves overlaid as black dots. Note: Regions with little difference ($<10^{-7}$) are plotted as white areas since they become Not-a-Number (NaN) in log scale. (*fig5*)

</div>
<div id="div_fig2">

<img src="tmp_2405.03135/./fig6-muarea.png" alt="Fig6" width="100%"/>

**Figure 6. -** Total area as a function of the difference between the recovered and fiducial magnification maps. Left: comparison between Lenstool(blue) and pixelated modeling (pink) with the best-fit result defined as model with minimum $\chi^2$; right: same with left but with the best-fit result defined as medians of the parameters.  (*fig6*)

</div>
<div id="div_fig3">

<img src="tmp_2405.03135/./fig4-mass.png" alt="Fig3" width="100%"/>

**Figure 3. -** Circularly averaged mass profiles of the cluster recovered with blue: Lenstool and red: pixelated modeling. Shaded regions are obtained from the randomly selected 2000 realizations in the MCMC outputs. The input mass profile is indicated as the black line. (*fig4*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2405.03135"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\dmnote}[1]{{\bf \color{blue}[D.M.: #1]}}$
$\newcommand{\dmfnote}[1]{{\bf \color{orange}[D.M.F.: #1]}}$
$\newcommand{\hknote}[1]{{\bf \color{green}[H.K.: #1]}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\dmnote}[1]{{\bf \color{blue}[D.M.: #1]}}$
$\newcommand{\dmfnote}[1]{{\bf \color{orange}[D.M.F.: #1]}}$
$\newcommand{\hknote}[1]{{\bf \color{green}[H.K.: #1]}}$</div>



<div id="title">

# Three-temperature radiation hydrodynamics with PLUTO

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2405.03375-b31b1b.svg)](https://arxiv.org/abs/2405.03375)<mark>Appeared on: 2024-05-07</mark> -  _Accepted to Astronomy and Astrophysics; 22 pages, 19 figures incl. Appendix. Comments and questions welcome_

</div>
<div id="authors">

<mark>D. Muley</mark>, J. D. M. Fuksman, <mark>H. Klahr</mark>

</div>
<div id="abstract">

**Abstract:** In circumstellar disks around young stars, the gravitational influence of nascent planets produces telltale patterns in density, temperature, and kinematics. To better understand these signatures, we first performed 3D hydrodynamical simulations of a 0.012 $M_{\odot}$ disk, with a Saturn-mass planet orbiting circularly in-plane at 40 au. We tested four different disk thermodynamic prescriptions (in increasing order of complexity, local isothermality, $\beta$ -cooling, two-temperature radiation hydrodynamics, and three-temperature radiation hydrodynamics), finding that $\beta$ -cooling offers a reasonable approximation for the three-temperature approach when the planet is not massive or luminous enough to substantially alter the background temperature and density structure. Thereafter, using the three-temperature scheme, we relaxed this assumption, simulating a range of different planet masses (Neptune-mass, Saturn-mass, Jupiter-mass) and accretion luminosities (0, $10^{-3} L_{\odot}$ ) in the same disk. Our investigation revealed that signatures of disk-planet interaction strengthen with increasing planet mass, with circumplanetary flows becoming prominent in the high-planet-mass regime. Accretion luminosity, which adds pressure support around the planet, was found to weaken the midplane Doppler-flip, potentially visible in optically thin tracers like C $^{18}$ O, while strengthening the spiral signature, particularly in upper disk layers sensitive to thicker lines, like those of $^{12}$ CO.

</div>

<div id="div_fig1">

<img src="tmp_2405.03375/./azi_cuts_integrated_spiral_cut_allmass_3t_scalar.png" alt="Fig17.1" width="50%"/><img src="tmp_2405.03375/./azi_cuts_integrated_spiral_cut_allmass_3t_vector.png" alt="Fig17.2" width="50%"/>

**Figure 17. -** Measurement of the azimuthal perturbation in each normalized quantity $x$, averaged over an azimuthal range of $\phi_{\rm peak} \pm 2 h_p$ according to Equation \ref{eq:spiral_avg}. Densities and temperatures are plotted in the left panel while velocities are in the right. Open circles indicate non-accreting planets while closed circles indicate those with accretion luminosity (see Section \ref{sec:acclum_analysis} for discussion). At $z/r \approx \theta = 0.2$, where dust and gas are not well-coupled, the plotted $T_{\rm dust}$ amplitude reflects the radial "pseudo-arm" (see discussion in Section \ref{sec:equilibrium_temp}) rather than the Lindblad spiral. (*fig:spiral_amplitude_quantitative*)

</div>
<div id="div_fig2">

<img src="tmp_2405.03375/./vx1_diff_spiral_cut_allmass_3t.png" alt="Fig18" width="100%"/>

**Figure 18. -** Radial velocities $v_r$ from 3T, zero-accretion-luminosity simulations with Neptune-mass _(top row)_, Saturn-mass _(middle row)_, and Jupiter-mass _(bottom row)_ planets, at various altitudes in the disk _(left, middle, right columns)_. Grey circles indicate the planetary Hill radius. All $v_r$ values are expressed as a function of initial isothermal sound speed at the location of the planet, $c_{\rm iso, p} = \sqrt{p_{p,0}/\rho_{p, 0}}$.  (*fig:rad_vel_perturbation*)

</div>
<div id="div_fig3">

<img src="tmp_2405.03375/./azi_cuts_integrated_spiral_cut_allmass_3t_scalar.png" alt="Fig29.1" width="50%"/><img src="tmp_2405.03375/./azi_cuts_integrated_spiral_cut_allmass_3t_vector.png" alt="Fig29.2" width="50%"/>

**Figure 29. -** Measurement of the azimuthal perturbation in each normalized quantity $x$, averaged over an azimuthal range of $\phi_{\rm peak} \pm 2 h_p$ according to Equation \ref{eq:spiral_avg}. Densities and temperatures are plotted in the left panel while velocities are in the right. Open circles indicate non-accreting planets while closed circles indicate those with accretion luminosity (see Section \ref{sec:acclum_analysis} for discussion). At $z/r \approx \theta = 0.2$, where dust and gas are not well-coupled, the plotted $T_{\rm dust}$ amplitude reflects the radial "pseudo-arm" (see discussion in Section \ref{sec:equilibrium_temp}) rather than the Lindblad spiral. (*fig:spiral_amplitude_quantitative*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2405.03375"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

414  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
